# Stock Data Analysis

### Import Required Packages

In [8]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display


### API KEY VALIDITY CHECK

In [9]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [10]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

### BUILDING URL AND RESPONSE STRUCTIRE

In [11]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"
    
    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_three_year_ago = datetime.datetime.now() - datetime.timedelta(days=1095)
        params = {"frequency": "1wk", "period1": datetime_three_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.title.string if soup.title else "No title found"

        for irrelevant in soup.body(['script', 'style', 'image', 'input']):
            irrelevant.decompose()

        table_data = soup.find('table')

        return title, table_data



### BUILDNIG SYSTEM AND USER PROMPTS

In [12]:
def build_stock_analysis_prompt(stock_symbol, table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over three years timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."

    return [
        {'role':'system', "content":sys_prompt},
        {"role":"user", "content":user_prompt}
    ]

### AGGREGATING EVERYTHING WITH OPENAI

In [13]:
def analyse_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, table_data = stock_data_page.get_stock_data()
    openai = OpenAI()
    response = openai.chat.completions.create(
        model= "gpt-4o-mini",
        messages= build_stock_analysis_prompt(stock_symbol, table_data)
    )

    return response.choices[0].message.content

In [14]:
def display_analysis(stock_symbol):
    display(Markdown(analyse_stock_trends(stock_symbol)))

### FINAL RESPONSE

In [15]:
display_analysis('GOOG')

## Analysis of GOOG Stock Data

Below is an analysis of the Google (GOOG) stock based on the provided data over recent weeks. 

### Key Metrics from the Data

1. **Price Movement** (Date: 2022 - 2025):
   - **Starting Price**: The lowest recorded price shown is around \$89.31 in December 2022.
   - **Ending Price**: The most recent price on February 21, 2025 is \$181.58.

2. **Price Overview**:
   - **Open Prices**: 
     - The opening prices show fluctuations, starting from \$89.83 in January 2023, rising to \$202.21 by February 2025, showing significant growth.
   - **Closing Prices**: 
     - Closing prices reflect a general uptrend from approximately \$88.73 in December 2022 to around \$181.58.
 
3. **Highs and Lows**: 
   - **High Price**: The highest recorded price in the data is \$208.70 in February 2025.
   - **Low Price**: The lowest price recorded is \$85.57, which occurred in January 2023.

4. **Trading Volume**: 
   - There is high trading volume in the data, peaking at volumes exceeding 200 million shares traded in some weeks. This suggests strong market interest and liquidity.

### Trends Observed

1. **Upward Trend**: 
   - Over the analyzed timeframe, the stock has shown a clear upward trend. Starting below \$100 in January 2023, GOOG has significantly appreciated in value.
   - In general, there appears to be a strong recovery from a low point following a larger market correction, resulting in a substantial rebound.

2. **Volatility**:
   - Throughout the data, there are multiple instances where the stock shows volatility (e.g., significant drops followed by recoveries). This indicates sensitivity to market conditions.
   - High volatility in trading volumes during these periods suggests changes in trader sentiment and possibly external factors affecting GOOG.

### Total Returns Calculation

To calculate total returns percentage over this period, we can use the formula:

\[
\text{Total Return} (\%) = \left(\frac{\text{Ending Value} - \text{Beginning Value}}{\text{Beginning Value}}\right) \times 100
\]

Here:
- **Ending Value** = \$181.58 (on February 21, 2025)
- **Beginning Value** = \$89.31 (on December 26, 2022)

Now, calculate the return:

\[
\text{Total Return} (\%) = \left(\frac{181.58 - 89.31}{89.31}\right) \times 100
\]

\[
\text{Total Return} (\%) = \left(\frac{92.27}{89.31}\right) \times 100 \approx 103.4\%
\]

### Summary

- **Overall Performance**: GOOG has shown robust growth of approximately **103.4%** from late 2022 to early 2025.
- **Investment Consideration**: Investors could view this substantial return as a sign of strong performance, though they should remain mindful of the stock's volatility. It is also important to consider external factors that might impact the tech industry and Google's business model in the future.

This summary provides a backdrop of Google's recent performance, helping investors make informed decisions.

## OLLAMA INSTEAD OF OPENAI

In [16]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [17]:
messages = [
    {"role":"user", "content":"Describe some of the Technical applicaitons of TLS"}
]

In [18]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [19]:
payload = {
    "model" : MODEL,
    "messages": messages,
    "stream":False
}

In [ ]:

def analyse_stock_trends_ollama(stock_symbol):
    stock_data_package =YahooFinanceWebsite(stock_symbol)
    title, table_data = stock_data_package.get_stock_data()
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    print(response.json()['message']['content'])



In [23]:
analyse_stock_trends_ollama("GOOG")

{'error': 'model requires more system memory (3.5 GiB) than is available (2.8 GiB)'}
